# Data cleaning of zomato dataset

## Alert: Because this is cleaning part, you'll see lots of words like 'Removing','Replacing' and 'Splitting'.

In [150]:
import pandas as pd

In [151]:
df = pd.read_csv('zomato_vadodara.csv')#csv-file we achived by using beautiful soup

In [152]:
df.head(3)

,Restaurant,Cousines,Rating,Rating_counts,Delivery_rating,Delivery_rating_counts,Area,Adress,Price_for_2,Open_time
0,Jassi De Parathe,North Indian,4.7,"(1,634)",4.2,(18.5K),Alkapuri,"R.C Dutt Road, Near Kalaniketan, Alkapuri, Vad...",₹600,11am – 11:30pm (Mon-Sun)
1,Marwari Food Corner,"North Indian, Rajasthani, Fast Food, Chinese",4.3,"(1,505)",3.8,(29.2K),Karelibaug,"Shop No. 21 & 22 Ratri-bazaar, Mangal Pandey R...",₹250,4pm – 2am (Mon-Sun)
2,KFC,"Burger, Fast Food, Finger Food, Beverages",4.2,"(1,003)",4.3,(16.8K),"Vadodara Central, Alkapuri","19, Ground Floor, Vadodara Central Mall, Near ...",₹450,1pm – 9:30pm (Mon-Sun)


In [153]:
df.dtypes

Restaurant                 object
Cousines                   object
Rating                    float64
Rating_counts              object
Delivery_rating           float64
Delivery_rating_counts     object
Area                       object
Adress                     object
Price_for_2                object
Open_time                  object
dtype: object

In [154]:
list_=['Rating_counts','Delivery_rating_counts']

### Removing '()' from Rating counts and Delivery Rating Counts

In [155]:
for x in list_:
    df[x]=df[x].str.strip('()')

### Removing (Reviews) text  which is right after some values in Delivery Rating Counts  

In [156]:
df['Delivery_rating_counts']= df['Delivery_rating_counts'].str.split(' ',expand=True)[0]

### Removing unnecessary comma in rating and delivery rating counts like 1,000~1000

In [157]:
for x in list_:
    df[x]=df[x].str.replace(',','')

In [158]:
df.head(3)

,Restaurant,Cousines,Rating,Rating_counts,Delivery_rating,Delivery_rating_counts,Area,Adress,Price_for_2,Open_time
0,Jassi De Parathe,North Indian,4.7,1634,4.2,18.5K,Alkapuri,"R.C Dutt Road, Near Kalaniketan, Alkapuri, Vad...",₹600,11am – 11:30pm (Mon-Sun)
1,Marwari Food Corner,"North Indian, Rajasthani, Fast Food, Chinese",4.3,1505,3.8,29.2K,Karelibaug,"Shop No. 21 & 22 Ratri-bazaar, Mangal Pandey R...",₹250,4pm – 2am (Mon-Sun)
2,KFC,"Burger, Fast Food, Finger Food, Beverages",4.2,1003,4.3,16.8K,"Vadodara Central, Alkapuri","19, Ground Floor, Vadodara Central Mall, Near ...",₹450,1pm – 9:30pm (Mon-Sun)


### We need to change 'K' into column of 1000s and them multiply that column with counts column 

In [159]:
df['K']=df['Delivery_rating_counts'].str.extract(r'[\d\.]+([K]+)').fillna(1).replace('K',1000)

In [160]:
df['Delivery_rating_counts']=df['Delivery_rating_counts'].replace(r'[K]+$','',regex=True).astype(float)

In [161]:
df['Delivery_rating_counts']= df['Delivery_rating_counts']*df['K']


In [162]:
df.head(2) #Done

,Restaurant,Cousines,Rating,Rating_counts,Delivery_rating,Delivery_rating_counts,Area,Adress,Price_for_2,Open_time,K
0,Jassi De Parathe,North Indian,4.7,1634,4.2,18500.0,Alkapuri,"R.C Dutt Road, Near Kalaniketan, Alkapuri, Vad...",₹600,11am – 11:30pm (Mon-Sun),1000
1,Marwari Food Corner,"North Indian, Rajasthani, Fast Food, Chinese",4.3,1505,3.8,29200.0,Karelibaug,"Shop No. 21 & 22 Ratri-bazaar, Mangal Pandey R...",₹250,4pm – 2am (Mon-Sun),1000


### Removing unncessary spaces

In [163]:
df['Rating_counts']= df['Rating_counts'].str.split(' ',expand=True)[0]
df['Price_for_2']= df['Price_for_2'].str.replace(',','')
df['Rating_counts']=df['Rating_counts'].astype(float)

### Replacing  '₹' with empty space

In [164]:
df['Price_for_2']=df['Price_for_2'].replace('₹','',regex=True).astype(int)

In [165]:
df['Area'].unique()

array(['Alkapuri', 'Karelibaug', 'Vadodara Central, Alkapuri', 'Mandvi',
       'Nizampura', 'Kendranagar', 'Vadiwadi', 'Akota', 'Makarpura',
       'Diwalipura', 'Sayajigunj', 'Fatehgunj', 'Manjalpur', 'Gotri',
       'Subhanpura', 'Gorwa', 'Bhayli', 'Inorbit Mall, Subhanpura',
       'Suryanagar', 'Sayajipura', 'Sama', 'Panchvati',
       'Sayaji Hotel, Sayajiganj', 'Lilleria Food Court, Vadiwadi',
       'Tarsali', 'M Cube Mall, Vadiwadi',
       'Hotel Express Towers, Alkapuri', 'Navapura', 'L&T Knowledge City',
       'Vivanta Vadodara, Akota', 'Yakutpura',
       'Seven Seas Mall, Fatehgunj'], dtype=object)

### We have some multiple values for same areas, we needs to fix this

In [166]:
df['area'] = df['Area'].str.split(',',expand=True)[1].str.replace(' ','')
area = df['area'].unique()
for i in area:
    index = df[df.loc[:,'area']==i].index
    df.loc[index,'Area']=i

In [167]:
df['Area'].value_counts()

Alkapuri              90
Diwalipura            64
Karelibaug            50
Vadiwadi              50
Akota                 42
Fatehgunj             38
Sayajigunj            29
Subhanpura            29
Kendranagar           27
Mandvi                26
Manjalpur             23
Nizampura             20
Gotri                 19
Sayajipura            10
Makarpura             10
Sama                   6
Suryanagar             4
Navapura               3
Bhayli                 3
Sayajiganj             2
Gorwa                  2
Panchvati              1
Tarsali                1
Yakutpura              1
L&T Knowledge City     1
Name: Area, dtype: int64

In [168]:
df[df['Area']=='Sayajiganj'].index

Int64Index([238, 516], dtype='int64')

In [169]:
df.loc[df[df['Area']=='Sayajiganj'].index,'Area']='Sayajigunj'

In [170]:
df.drop('K',axis=1,inplace=True)

## Now the Hardest part, changing time range into date-time pandas series

### To be honest this took me half a day to figured out. Because we don't have year and date column and also we have range of time (not specific value of time). But eventually hard work paid off

### Splitting Open time column into opening hours and closing hours

In [171]:
df[['time','week']] = df['Open_time'].str.split('(',1,expand=True)

df[['1st_time','2nd_time','3rd_time']] = df['time'].str.split(',',expand=True)

df[['1st_time_open','1st_time_close']]=df['1st_time'].str.split('–',expand=True)

df[['2nd_time_open','2nd_time_close']]=df['2nd_time'].str.split('–',expand=True)

### Removing empty spaces

In [172]:
df['1st_time_open'] = df['1st_time_open'].str.replace(' ','')
df['2nd_time_open'] = df['2nd_time_open'].str.replace(' ','')

df['1st_time_close'] = df['1st_time_close'].str.replace(' ','')
df['2nd_time_close'] = df['2nd_time_close'].str.replace(' ','')

### Computer's brain can't understand words, Therefore we need to fix words like 'noon' and 'midnight'.

In [173]:
df.loc[df[df.loc[:,'1st_time_open']=='12noon'].index,'1st_time_open']='12pm'
df.loc[df[df.loc[:,'1st_time_open']=='24Hours'].index,'1st_time_open']='12am'
df.loc[df[df.loc[:,'1st_time_open']=='12midnight'].index,'1st_time_open']='12am'

df.loc[df[df.loc[:,'1st_time_close']=='12midnight'].index,'1st_time_close']='12am'

In [174]:
df.loc[47,'1st_time_open']='12pm'
df.loc[51,'1st_time_open']='11pm'
df.loc[161,'1st_time_open']='6pm'
df.loc[322,'1st_time_open']='1pm'
df.loc[404,'1st_time_open']='9am'

df.loc[47,'2nd_time_open']='7pm'
df.loc[47,'2nd_time_close']='10:30pm'

df.loc[47,'1st_time_close']='3pm'
df.loc[51,'1st_time_close']='12pm'
df.loc[161,'1st_time_close']='11pm'
df.loc[322,'1st_time_close']='12am'
df.loc[404,'1st_time_close']='9am'

In [175]:
df.loc[df[df.loc[:,'2nd_time_open']=='12midnight'].index,'2nd_time_open']='12am'
df.loc[df[df.loc[:,'2nd_time_open']=='12noon'].index,'2nd_time_open']='12pm'

df.loc[df[df.loc[:,'2nd_time_close']=='12midnight'].index,'2nd_time_close']='12am'
df.loc[df[df.loc[:,'2nd_time_close']=='12midnight...'].index,'2nd_time_close']='12am'

### Another Problem, we need to change the format 11am to 11:00am

In [176]:
df['1st_time_open_ampm'] = df['1st_time_open'].str.extract(r'[\d\.]+([am|pm]+)')
df['1st_time_open']=df['1st_time_open'].replace(r'[am|pm]+$','',regex=True)
df[['1st_time_open_hour','1st_time_open_minute']]=df['1st_time_open'].str.split(':',expand=True)
df['1st_time_open_minute'] =df['1st_time_open_minute'].fillna('00')
df['1st_time_open']=df['1st_time_open_hour']+':'+df['1st_time_open_minute']+df['1st_time_open_ampm']

In [177]:
df['1st_time_close_ampm'] = df['1st_time_close'].str.extract(r'[\d\.]+([am|pm]+)')
df['1st_time_close']=df['1st_time_close'].replace(r'[am|pm]+$','',regex=True)
df[['1st_time_close_hour','1st_time_close_minute']]=df['1st_time_close'].str.split(':',expand=True)
df['1st_time_close_minute'] =df['1st_time_close_minute'].fillna('00')
df['1st_time_close']=df['1st_time_close_hour']+':'+df['1st_time_close_minute']+df['1st_time_close_ampm']

In [178]:
df['2nd_time_open_ampm'] = df['2nd_time_open'].str.extract(r'[\d\.]+([am|pm]+)')
df['2nd_time_open']=df['2nd_time_open'].replace(r'[am|pm]+$','',regex=True)
df[['2nd_time_open_hour','2nd_time_open_minute']]=df['2nd_time_open'].str.split(':',expand=True)
df['2nd_time_open_minute'] =df['2nd_time_open_minute'].fillna('00')
df['2nd_time_open']=df['2nd_time_open_hour']+':'+df['2nd_time_open_minute']+df['2nd_time_open_ampm']

In [179]:
df['2nd_time_close_ampm'] = df['2nd_time_close'].str.extract(r'[\d\.]+([am|pm]+)')
df['2nd_time_close']=df['2nd_time_close'].replace(r'[am|pm]+$','',regex=True)
df[['2nd_time_close_hour','2nd_time_close_minute']]=df['2nd_time_close'].str.split(':',expand=True)
df['2nd_time_close_minute'] =df['2nd_time_close_minute'].fillna('00')
df['2nd_time_close']=df['2nd_time_close_hour']+':'+df['2nd_time_close_minute']+df['2nd_time_close_ampm']

### Done Now we have all hours in one pattern, then next problem is we don't have year and date in our time like column, After putting so much time, I realized that answer is simple af. Just add dummy year and date who cares! Although we will be using only hours.

In [180]:
df['1st_time_open']='2020-10-15'+' '+df['1st_time_open']
df['1st_time_close']='2020-10-15'+' '+df['1st_time_close']

df['2nd_time_open']='2020-10-15'+' '+df['2nd_time_open']
df['2nd_time_close']='2020-10-15'+' '+df['2nd_time_close']

### Changing into date-time pandas series 

In [181]:
df['1st_time_open']=pd.to_datetime(df['1st_time_open'])
df['1st_time_close']=pd.to_datetime(df['1st_time_close'])

df['2nd_time_open']=pd.to_datetime(df['2nd_time_open'])
df['2nd_time_close']=pd.to_datetime(df['2nd_time_close'])

In [182]:
df['1st_time_open'].head()

0   2020-10-15 11:00:00
1   2020-10-15 16:00:00
2   2020-10-15 13:00:00
3   2020-10-15 11:00:00
4   2020-10-15 09:00:00
Name: 1st_time_open, dtype: datetime64[ns]

### Hell Yeah! 

In [183]:
df['week'] = df['week'].str.replace(' ','')
df['week'].value_counts().head(7)

Mon-Sun)                           506
Mon-Sat),Closed(Sun)                 3
Mon,Wed,Thu,Fri,Sat,Sun)...          2
Mon,Tue,Wed...                       2
Mon),11am–3pm,7pm–11pm(Tue-Sun)      2
Mon),6pm–11pm(Tue-Sun)               1
Mon),12noon–11pm(Tue-Sun)            1
Name: week, dtype: int64

### Keeping necessary columns 

In [184]:
df = df[['Restaurant','Cousines','Rating','Rating_counts','Delivery_rating','Delivery_rating_counts','Area','Adress','Price_for_2','1st_time_open','1st_time_close','2nd_time_open','2nd_time_close','week']]

In [185]:
df.head(3)

,Restaurant,Cousines,Rating,Rating_counts,Delivery_rating,Delivery_rating_counts,Area,Adress,Price_for_2,1st_time_open,1st_time_close,2nd_time_open,2nd_time_close,week
0,Jassi De Parathe,North Indian,4.7,1634.0,4.2,18500.0,Alkapuri,"R.C Dutt Road, Near Kalaniketan, Alkapuri, Vad...",600,2020-10-15 11:00:00,2020-10-15 23:30:00,NaT,NaT,Mon-Sun)
1,Marwari Food Corner,"North Indian, Rajasthani, Fast Food, Chinese",4.3,1505.0,3.8,29200.0,Karelibaug,"Shop No. 21 & 22 Ratri-bazaar, Mangal Pandey R...",250,2020-10-15 16:00:00,2020-10-15 02:00:00,NaT,NaT,Mon-Sun)
2,KFC,"Burger, Fast Food, Finger Food, Beverages",4.2,1003.0,4.3,16800.0,Alkapuri,"19, Ground Floor, Vadodara Central Mall, Near ...",450,2020-10-15 13:00:00,2020-10-15 21:30:00,NaT,NaT,Mon-Sun)


### converting neat and clean pandas dataframe into csv file 

In [186]:
df.to_csv('Zomato_cleaned.csv',index=False)

### Check out next notebook, notebook of this csv file's analysis and insights